In [4]:
import json
import numpy as np
from functools import partial
from collections import Counter

In [5]:
with open("/home/hoyeung/blob_matterport3d/asknav/asknav_train_original.json") as f:
    data = json.load(f)

In [6]:
len(data)

94798

In [7]:
data[0].keys()

dict_keys(['object_indices', 'scan', 'object_name', 'path_id', 'trajectories', 'end_regions', 'paths', 'instructions', 'start_region_name', 'heading', 'end_region_name', 'start_region', 'distances'])

In [8]:
# most common of all
instr_set = set()
instr_list = []
for d in data:
    instr_set.add(d['instructions'][0])
    instr_list.append(d['instructions'][0])

len(instr_set)

1444

In [9]:
Counter(instr_list).most_common(10)

[('find a sink in the kitchen .', 650),
 ('find a refrigerator in the kitchen .', 626),
 ('find a couch in the living room .', 614),
 ('find a mirror in one of the bedrooms .', 589),
 ('find a picture in the living room .', 586),
 ('find a table in the living room .', 579),
 ('find a chair in one of the bedrooms .', 566),
 ('find a nightstand in one of the bedrooms .', 560),
 ('find a kitchen counter in the kitchen .', 533),
 ('find a lamp in one of the bedrooms .', 513)]

In [10]:
# Figure 9 number of actions 5-9
def filter_action_lens(low, high, d):
    mean_path_len = np.mean([len(path) for path in d['paths']])
    if low <= mean_path_len and high >= mean_path_len:
        return True
    else:
        return False

filter_5_9 = partial(filter_action_lens, 5, 9)

# -------------------------------------------------------
# Figure 5 start rooms
def filter_start_room(start_rm_list, d):
    if d['start_region_name'] in start_rm_list:
        return True
    else:
        return False
    
filter_start_hallway = partial(filter_start_room, ['hallway'])

# -------------------------------------------------------
# Figure 5 goal rooms
def filter_goal_room(goal_rm_list, d):
    if d['end_region_name'] in goal_rm_list:
        return True
    else:
        return False
    
filter_end_bedroom = partial(filter_goal_room, ['bedroom'])
filter_end_bathroom = partial(filter_goal_room, ['bathroom'])
filter_end_common = partial(filter_goal_room, ['bedroom', 'bathroom'])
# -------------------------------------------------------
# Figure 4 objects
def filter_object(object_list, d):
    if d['object_name'] in object_list:
        return True
    else:
        return False
    
filter_picture = partial(filter_object, ['picture'])
filter_mirror = partial(filter_object, ['mirror'])
filter_pillow = partial(filter_object, ['pillow'])
filter_picture_mirror_pillow = partial(filter_object, ['mirror', 'picture', 'pillow'])
# -------------------------------------------------------
# Language instruction
def filter_instruction(instr_str_list, d):
    for instr_str in instr_str_list:
        if instr_str in d['instructions'][0]:
            return True
    return False

filter_instruction_most_common =  partial(filter_instruction, ['find a sink in the kitchen '])
# -------------------------------------------------------
def filter_simple_case(d):
    if filter_pillow(d) and filter_5_9(d) and filter_start_hallway(d) and filter_end_bedroom(d):
        return True
    else:
        return False
    
def filter_simple_case2(d):
    if filter_mirror(d) and filter_5_9(d) and filter_start_hallway(d) and filter_end_bathroom(d):
        return True
    else:
        return False

def filter_common1(d):
    if filter_picture_mirror_pillow(d) and filter_5_9(d) and filter_start_hallway(d) and filter_end_bathroom(d):
        return True
    else:
        return False
    
def filter_common2(d):
    if filter_picture_mirror_pillow(d) and filter_5_9(d) and filter_start_hallway(d) and filter_end_common(d):
        return True
    else:
        return False
    
def filter_common3(d):
    if filter_instruction_most_common(d) and filter_5_9(d):
        return True
    else:
        return False

In [11]:
# 5-9, start hallway, end bedroom, pillow
len(list(filter(filter_simple_case, data)))

26

In [12]:
# 5-9, start hallway, end bathroom, mirror
len(list(filter(filter_simple_case2, data)))

34

In [13]:
# 5-9, start hallway, end bathroom, mirror/picture/pillow
len(list(filter(filter_common1, data)))

70

In [14]:
# 5-9, start hallway, end bathroom, mirror/picture/pillow
len(list(filter(filter_common2, data)))

173

In [15]:
# 5-9, 'find a sink in the kitchen '
len(list(filter(filter_common3, data)))

372

In [16]:
# any length, 'find a sink in the kitchen '
len(list(filter(filter_instruction_most_common, data)))

650

# extract by indices

In [17]:
# 5-9, 'find a sink in the kitchen '
data_common3 = [data[idx] for idx in range(len(data)) if filter_common3(data[idx])]
data_common3_idx = [idx for idx in range(len(data)) if filter_common3(data[idx])]

In [76]:
scans_common3 = [d['scan'] for d in data_common3]
Counter(scans_common3).most_common(10)

[('ac26ZMwG7aT', 38),
 ('8WUmhLawc2A', 37),
 ('cV4RVeZvu5T', 26),
 ('JmbYfDe2QKZ', 26),
 ('mJXqzFtmKg4', 23),
 ('VzqfbhrpDEA', 23),
 ('EDJbREhghzL', 22),
 ('82sE5b5pLXE', 22),
 ('gTV8FGcVJC9', 21),
 ('e9zR4mvMWw7', 21)]

In [78]:
examine_d = [d for d in data_common3 if d['scan'] == 'ac26ZMwG7aT']

In [81]:
print (examine_d[0]['scan'])
print (examine_d[37]['scan'])

ac26ZMwG7aT
ac26ZMwG7aT


In [114]:
# [d['path_id'] for d in examine_d]

In [112]:
examine_d[0]

{'distances': [17.83902011259488],
 'end_region_name': 'kitchen',
 'end_regions': [7],
 'heading': 5.235987755982989,
 'instructions': ['find a sink in the kitchen .'],
 'object_indices': [[348, 355]],
 'object_name': 'sink',
 'path_id': 156477,
 'paths': [['13faa4c86406405ba802f5bdcba744ee',
   '11fae969386e4e4fa6ed73ac9f20cd1d',
   '735cc8eae07743a1aa5d41ba878b0eb4',
   '057625814959401f8e969f5bfbf6cca0',
   '91b29545634c4754bfe914564d6c950f',
   'effea130a5984e53bd2bef2bbe4ba4c1',
   '57b84179a9a24ff49697d9349a21bc60']],
 'scan': 'ac26ZMwG7aT',
 'start_region': 25,
 'start_region_name': 'bedroom',
 'trajectories': [[[0, 1, 0],
   [0, 1, 0],
   [0, 1, 0],
   [0, 1, 0],
   [0, 1, 0],
   [1, 0, 0],
   [1, 0, 0],
   [0, -1, 0],
   [0, -1, 0],
   [1, 0, 0],
   [0, -1, 0],
   [1, 0, 0],
   [1, 0, 0],
   [0, 1, 0],
   [0, 1, 0],
   [2, 0, 0],
   [0, 0, 0]]]}

In [88]:
[d['paths'][0][-1] for d in examine_d]

['57b84179a9a24ff49697d9349a21bc60',
 '57b84179a9a24ff49697d9349a21bc60',
 '57b84179a9a24ff49697d9349a21bc60',
 '57b84179a9a24ff49697d9349a21bc60',
 '57b84179a9a24ff49697d9349a21bc60',
 '57b84179a9a24ff49697d9349a21bc60',
 '57b84179a9a24ff49697d9349a21bc60',
 '57b84179a9a24ff49697d9349a21bc60',
 '57b84179a9a24ff49697d9349a21bc60',
 '57b84179a9a24ff49697d9349a21bc60',
 '57b84179a9a24ff49697d9349a21bc60',
 '57b84179a9a24ff49697d9349a21bc60',
 '57b84179a9a24ff49697d9349a21bc60',
 '57b84179a9a24ff49697d9349a21bc60',
 '57b84179a9a24ff49697d9349a21bc60',
 '57b84179a9a24ff49697d9349a21bc60',
 '57b84179a9a24ff49697d9349a21bc60',
 '57b84179a9a24ff49697d9349a21bc60',
 '57b84179a9a24ff49697d9349a21bc60',
 '57b84179a9a24ff49697d9349a21bc60',
 '57b84179a9a24ff49697d9349a21bc60',
 '57b84179a9a24ff49697d9349a21bc60',
 '57b84179a9a24ff49697d9349a21bc60',
 '57b84179a9a24ff49697d9349a21bc60',
 '57b84179a9a24ff49697d9349a21bc60',
 '57b84179a9a24ff49697d9349a21bc60',
 '57b84179a9a24ff49697d9349a21bc60',
 

In [92]:
[len(d['paths'][0]) for d in examine_d]

[7,
 9,
 9,
 5,
 7,
 9,
 5,
 9,
 7,
 8,
 5,
 7,
 6,
 5,
 6,
 6,
 7,
 6,
 6,
 5,
 8,
 9,
 5,
 7,
 6,
 6,
 8,
 6,
 6,
 9,
 9,
 7,
 6,
 9,
 8,
 5,
 9,
 5]

## Single example dataset

In [99]:
type(data)

list

In [106]:
# scan 'ac26ZMwG7aT', length 5, 'find a sink in the kitchen '
examine_d[-1]

{'distances': [11.124175652960771],
 'end_region_name': 'kitchen',
 'end_regions': [7],
 'heading': 5.235987755982989,
 'instructions': ['find a sink in the kitchen .'],
 'object_indices': [[348, 355]],
 'object_name': 'sink',
 'path_id': 156482,
 'paths': [['828b8f02ffd145ac8b351cbb70682dc2',
   '7fb0cc6c0754461ca29084c1d793b600',
   '96b25d845ff447cc989dfd87c88ff8b8',
   'fbc5ba2686d24a2d86cc1ac0575325ae',
   '57b84179a9a24ff49697d9349a21bc60']],
 'scan': 'ac26ZMwG7aT',
 'start_region': 0,
 'start_region_name': 'other room',
 'trajectories': [[[0, 1, 0],
   [0, 1, 0],
   [0, 1, 0],
   [0, 1, 0],
   [0, 1, 0],
   [1, 0, 0],
   [1, 0, 0],
   [0, 1, 0],
   [1, 0, 0],
   [1, 0, 0],
   [0, 0, 0]]]}

In [107]:
with open("/home/hoyeung/blob_matterport3d/asknav/asknav_train_small_single_example.json", "w") as f:
    json.dump([examine_d[-1]]*100, f)

## Single house, short, same goal example dataset

In [94]:
# scan 'ac26ZMwG7aT', 5-9, 'find a sink in the kitchen '
len(examine_d)

38

In [104]:
type(examine_d)

list

In [105]:
with open("/home/hoyeung/blob_matterport3d/asknav/asknav_train_small_single_house_single_goal_short.json", "w") as f:
    json.dump(examine_d, f)

## Single house, any length, same goal example dataset

In [95]:
# any length, 'find a sink in the kitchen '
data_common4 = [data[idx] for idx in range(len(data)) if filter_instruction_most_common(data[idx])]
data_common4_idx = [idx for idx in range(len(data)) if filter_instruction_most_common(data[idx])]

In [97]:
# scan 'ac26ZMwG7aT', any length, 'find a sink in the kitchen '
examine_e = [d for d in data_common4 if d['scan'] == 'ac26ZMwG7aT']
len(examine_e)

59

## Three houses, short, same goal example dataset

In [95]:
# any length, 'find a sink in the kitchen '
data_common5 = [data[idx] for idx in range(len(data)) if filter_instruction_most_common(data[idx])]
data_common5_idx = [idx for idx in range(len(data)) if filter_instruction_most_common(data[idx])]

In [19]:
#  ('ac26ZMwG7aT', 38),
#  ('8WUmhLawc2A', 37),
#  ('cV4RVeZvu5T', 26)
#  3 houses, 5-9, 'find a sink in the kitchen '
examine_f = [d for d in data_common3 if d['scan'] in ['ac26ZMwG7aT', '8WUmhLawc2A', 'cV4RVeZvu5T']]
len(examine_f)

101

In [29]:
# short path length by vertex ID
len(examine_f[91]['paths'][0])

9

In [23]:
# long trajectory length by tota=l number of gold actions
len(examine_f[91]['trajectories'][0])

20

In [26]:
len(examine_f[90]['trajectories'][0])

13

In [24]:
len(examine_f[91]['trajectories'][1])

IndexError: list index out of range

In [118]:
with open("/home/hoyeung/blob_matterport3d/asknav/asknav_train_small_three_houses_single_goal_short.json", "w") as f:
    json.dump(examine_f, f)

## Multiple houses, any length, same goal example dataset

In [98]:
# any scans, any length, 'find a sink in the kitchen '
len(data_common4)

650

In [119]:
for _ in range(11): print("ok")

ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok


In [120]:
l = [[None]] * 10
l

[[None],
 [None],
 [None],
 [None],
 [None],
 [None],
 [None],
 [None],
 [None],
 [None]]

In [121]:
l[0].append(1)
l

[[None, 1],
 [None, 1],
 [None, 1],
 [None, 1],
 [None, 1],
 [None, 1],
 [None, 1],
 [None, 1],
 [None, 1],
 [None, 1]]

In [122]:
l = [[None]*3] * 10
l

[[None, None, None],
 [None, None, None],
 [None, None, None],
 [None, None, None],
 [None, None, None],
 [None, None, None],
 [None, None, None],
 [None, None, None],
 [None, None, None],
 [None, None, None]]

In [123]:
l[0].append(1)
l

[[None, None, None, 1],
 [None, None, None, 1],
 [None, None, None, 1],
 [None, None, None, 1],
 [None, None, None, 1],
 [None, None, None, 1],
 [None, None, None, 1],
 [None, None, None, 1],
 [None, None, None, 1],
 [None, None, None, 1]]

SyntaxError: invalid syntax (<ipython-input-124-cbdf10b85b89>, line 1)

In [125]:
l = [None] * 10

In [126]:
l

[None, None, None, None, None, None, None, None, None, None]

In [127]:
l[0] = 1
l

[1, None, None, None, None, None, None, None, None, None]

In [129]:
l = [[].copy()] * 10

In [130]:
l

[[], [], [], [], [], [], [], [], [], []]

In [131]:
l[0].append(1)
l

[[1], [1], [1], [1], [1], [1], [1], [1], [1], [1]]

In [133]:
l = [[] for _ in range(10)]
l

[[], [], [], [], [], [], [], [], [], []]

In [134]:
l[0] = 1

In [135]:
l

[1, [], [], [], [], [], [], [], [], []]

In [139]:
d = [[None for s in range(3)] for _ in range(10)]
d

[[None, None, None],
 [None, None, None],
 [None, None, None],
 [None, None, None],
 [None, None, None],
 [None, None, None],
 [None, None, None],
 [None, None, None],
 [None, None, None],
 [None, None, None]]

In [140]:
d[0][1] = [1,1,1]
d

[[None, [1, 1, 1], None],
 [None, None, None],
 [None, None, None],
 [None, None, None],
 [None, None, None],
 [None, None, None],
 [None, None, None],
 [None, None, None],
 [None, None, None],
 [None, None, None]]

In [ ]:
np.ones(len(obs), len(viewix_next_vertex_map[0])) * 1e9

## Sort data by length

In [30]:
# are all datapts are sorted within itself by path lengths?
more_than_one_path_data = []
for d in data:
    if len(d['paths']) > 1:
        more_than_one_path_data.append(d)

In [31]:
len(more_than_one_path_data)

26734

In [39]:
not_sorted = False
for d in more_than_one_path_data:
    path_lens = [len(p) for p in d['paths']]
    if sorted(path_lens) != path_lens:
        not_sorted = True
        print (path_lens)

[12, 10]
[13, 4]
[9, 8]
[12, 11]
[11, 8, 10]
[10, 8]
[6, 5, 8]
[8, 9, 8, 9]
[9, 8, 5, 10]
[5, 6, 5]
[13, 12]
[9, 4]
[14, 5, 3, 4, 5, 16, 15]
[6, 6, 4, 7, 6, 8]
[4, 3]
[10, 9]
[8, 7, 11]
[7, 8, 7, 7]
[8, 9, 8]
[6, 5]
[7, 10, 9, 7, 6]
[9, 8, 9, 8]
[6, 5]
[9, 8, 8]
[5, 6, 4]
[8, 7]
[6, 3]
[11, 15, 10]
[13, 12]
[8, 3, 3]
[12, 11, 12]
[3, 4, 8, 5]
[13, 9, 9, 6, 7, 9, 12, 15, 9]
[6, 5]
[6, 5, 8]
[11, 12, 12, 11, 11, 12]
[14, 6]
[10, 6]
[8, 6]
[8, 9, 7]
[9, 8]
[9, 9, 10, 7]
[8, 5]
[11, 10]
[14, 14, 13]
[14, 12]
[9, 8]
[8, 7, 3]
[11, 3]
[8, 7]
[5, 4, 6, 6]
[8, 8, 10, 10, 9]
[5, 3]
[8, 6, 7]
[12, 8]
[12, 12, 9, 9]
[7, 6, 5, 6, 7]
[6, 7, 6, 5]
[7, 6]
[7, 8, 5]
[6, 6, 5]
[9, 8]
[9, 9, 10, 8, 9]
[6, 4]
[4, 6, 4, 3]
[11, 9, 9, 8]
[11, 12, 11, 11, 12, 11]
[7, 7, 6]
[9, 8]
[14, 13, 14]
[15, 13]
[12, 11, 10]
[6, 8, 9, 6, 8]
[6, 5]
[6, 9, 10, 8]
[10, 8, 8, 7]
[10, 10, 11, 9]
[8, 7, 6]
[5, 5, 9, 5, 9]
[12, 6]
[8, 6]
[7, 6]
[4, 5, 3, 6]
[8, 9, 9, 7]
[11, 10]
[10, 9, 9, 9]
[9, 8, 8]
[9, 8]
[10, 9]
[6, 4]


[13, 8, 12]
[12, 6]
[10, 9]
[5, 3]
[12, 11]
[5, 4]
[8, 6, 10]
[14, 13, 13]
[5, 6, 12, 10, 11]
[14, 13]
[7, 7, 10, 6, 11, 10]
[9, 7, 8]
[7, 5, 5, 4]
[15, 9]
[10, 8]
[10, 9]
[11, 12, 10]
[4, 3]
[7, 6, 5, 7, 7]
[4, 3]
[11, 9, 8]
[6, 6, 6, 10, 10, 9, 9]
[9, 8]
[7, 5]
[14, 10, 12]
[5, 4]
[7, 6]
[12, 11]
[6, 5]
[12, 7]
[8, 7]
[14, 13]
[8, 7]
[12, 10, 13]
[10, 9]
[14, 13]
[9, 7, 12, 10]
[8, 7, 5]
[10, 7, 5]
[10, 9]
[8, 6, 4]
[14, 9, 7]
[10, 8]
[7, 8, 7]
[8, 5]
[11, 6]
[8, 6, 13, 12, 7]
[15, 14, 15, 14, 13, 13, 15]
[8, 5, 9, 7]
[5, 4]
[7, 6]
[8, 7]
[9, 7]
[5, 4, 6]
[7, 5]
[8, 7]
[6, 5, 4, 5, 5]
[7, 3]
[8, 8, 7, 7]
[12, 9]
[7, 6]
[10, 11, 10]
[12, 11]
[5, 7, 5]
[9, 7]
[14, 11]
[7, 6, 7, 7, 7]
[13, 9]
[10, 8, 9]
[5, 4]
[10, 14, 13, 12]
[4, 3, 3, 4, 4, 4]
[11, 9]
[5, 4]
[14, 7]
[9, 8]
[20, 18]
[9, 12, 9, 8]
[10, 11, 10, 11]
[8, 8, 5, 6]
[7, 6, 7]
[7, 5]
[9, 10, 9, 8]
[9, 8, 9, 10, 4, 5, 4, 3]
[10, 11, 8, 8, 9]
[8, 4]
[12, 11]
[4, 3, 4, 10, 9, 7, 8]
[5, 4, 5, 4, 4]
[12, 5]
[9, 9, 8]
[9, 8]
[3, 4, 

[8, 3]
[12, 13, 12, 11]
[9, 7, 10]
[6, 7, 5, 6]
[9, 8]
[10, 9]
[10, 10, 9, 10, 10]
[6, 3, 5]
[7, 9, 4]
[11, 10]
[11, 7, 8]
[8, 6]
[10, 9, 10]
[12, 4]
[9, 8]
[5, 6, 4, 5]
[13, 12]
[15, 16, 15]
[7, 6]
[11, 10]
[8, 7]
[10, 9]
[4, 5, 6, 5, 4, 5, 6, 5, 5]
[6, 7, 7, 6]
[9, 8, 9, 8]
[9, 13, 11]
[8, 7]
[15, 14]
[6, 4, 7]
[4, 11, 10, 8, 7]
[7, 7, 7, 8, 6]
[12, 10]
[6, 5]
[10, 9]
[7, 6]
[13, 5, 5]
[8, 8, 11, 9, 9, 9]
[8, 7]
[10, 12, 11, 11, 11]
[8, 7, 6]
[11, 10, 11, 11, 10]
[4, 3]
[10, 6]
[11, 5]
[8, 8, 11, 9, 9, 9]
[13, 9]
[7, 4]
[5, 4]
[8, 6]
[6, 7, 6, 6, 5]
[7, 6, 7]
[12, 10]
[13, 14, 16, 15]
[8, 6, 5, 7, 9]
[20, 11, 13]
[5, 4, 5, 13]
[11, 11, 6, 7, 7]
[12, 5]
[12, 6]
[5, 7, 6]
[15, 5]
[9, 11, 10, 10, 10]
[12, 3]
[11, 10]
[5, 4]
[7, 5, 12, 11, 6]
[8, 7]
[13, 11, 14]
[10, 9, 8]
[12, 12, 13, 11]
[13, 7]
[5, 4, 3]
[17, 16, 18, 17, 6, 5, 6, 14]
[6, 7, 7, 6]
[9, 8]
[13, 12]
[14, 17, 9, 6, 7, 14, 14]
[10, 11, 11, 10, 10, 11]
[8, 8, 9, 9, 8, 8, 8]
[10, 8]
[14, 12]
[10, 7, 7]
[6, 5]
[8, 8, 7, 7]
[4,

[4, 5, 4]
[6, 5, 5, 8]
[10, 8]
[10, 6]
[4, 3]
[9, 10, 9, 8, 9, 9]
[3, 4, 3, 4]
[11, 6]
[7, 6]
[6, 6, 5, 5]
[6, 5]
[10, 5]
[5, 6, 5, 5]
[10, 12, 12, 10, 11, 11, 11]
[4, 3]
[6, 5, 6, 5, 5]
[12, 8]
[10, 12, 9]
[9, 8]
[9, 10, 9, 7]
[7, 6]
[8, 7]
[10, 9]
[11, 10]
[8, 7]
[11, 4, 5]
[11, 11, 11, 10, 5]
[10, 8]
[13, 9, 8, 7, 13]
[4, 3]
[11, 10, 11, 10, 9, 9, 11]
[13, 12, 14, 13, 14, 14]
[13, 12]
[10, 9]
[7, 6]
[8, 6, 11, 10, 5]
[7, 7, 6]
[15, 14, 15, 15]
[8, 5, 5, 6, 4, 6, 10, 12, 14, 9, 11, 14]
[10, 9, 12]
[8, 7, 5, 5]
[8, 9, 11, 8, 8]
[6, 5]
[7, 9, 8]
[5, 5, 3]
[9, 8]
[8, 7, 8, 8, 7, 9, 7]
[10, 5, 4]
[7, 8, 8, 6]
[12, 11, 12, 12]
[7, 8, 4]
[5, 7, 6, 7, 8, 7, 7, 8]
[5, 4]
[11, 11, 10, 7]
[7, 6]
[5, 4]
[10, 12, 7]
[10, 9, 11, 11]
[17, 7]
[7, 10, 6, 8]
[9, 8]
[9, 8]
[10, 9]
[7, 5]
[6, 10, 9, 8]
[12, 11]
[13, 6, 9, 9, 7, 5, 13, 15, 17, 12, 14, 17]
[7, 6, 4, 16, 15, 13, 14]
[8, 4]
[11, 10]
[5, 4, 4, 3]
[11, 11, 10, 10]
[9, 6, 8, 5]
[8, 3]
[8, 7]
[11, 11, 10]
[7, 5]
[7, 5]
[6, 4]
[5, 5, 4, 4, 6, 1

[12, 13, 11]
[3, 5, 5, 4]
[8, 8, 7]
[11, 4, 7, 7, 5, 3, 11, 13, 15, 10, 12, 15]
[12, 11, 12]
[9, 8, 8, 8]
[7, 6]
[12, 10]
[8, 7]
[9, 8, 3, 3]
[11, 5]
[8, 7]
[8, 5]
[12, 13, 12, 11]
[5, 4, 5]
[15, 13, 7]
[8, 7, 8]
[7, 9, 9, 7]
[11, 12, 13, 12]
[11, 8, 10]
[7, 5]
[7, 6]
[4, 5, 4]
[11, 8, 6]
[14, 12, 9, 10, 10]
[6, 6, 8, 6, 6, 7]
[4, 6, 7, 5, 8, 7, 6, 8]
[9, 10, 8, 9, 7]
[6, 5, 3, 4]
[11, 10]
[11, 10, 11, 11, 10]
[10, 9]
[16, 15]
[7, 6]
[8, 7]
[7, 3]
[9, 8, 9]
[9, 9, 9, 10, 8]
[7, 6]
[12, 11, 12]
[8, 7]
[10, 9]
[12, 11]
[13, 11, 10]
[6, 7, 6]
[16, 18, 17, 17, 18]
[7, 6, 6]
[8, 7]
[7, 6]
[7, 8, 10, 9]
[11, 9]
[13, 14, 13, 14]
[14, 13]
[11, 9, 16, 13]
[15, 14]
[6, 5, 4]
[5, 3, 3]
[8, 9, 9, 5]
[11, 10, 10]
[10, 7, 6]
[13, 12]
[16, 15, 14, 15]
[6, 5]
[4, 4, 3]
[11, 7, 9, 8]
[7, 8, 7]
[3, 14, 13]
[19, 18, 15, 8]
[12, 11]
[4, 3]
[14, 13, 11]
[6, 5]
[7, 8, 8, 4, 5, 4, 3, 5, 3]
[10, 9]
[10, 7, 7, 7, 8]
[10, 11, 9]
[10, 9, 10, 11, 5, 6, 5, 6]
[13, 8]
[7, 6, 5]
[6, 5, 4]
[9, 11, 10]
[12, 8]
[7, 6]


[9, 8]
[9, 8, 5, 5]
[10, 9, 9, 9]
[13, 7]
[9, 5]
[14, 13]
[11, 7, 8]
[6, 5, 6, 6]
[12, 8]
[13, 8]
[13, 12]
[10, 9]
[6, 5, 5]
[8, 7]
[13, 13, 12]
[11, 12, 14, 11, 11]
[12, 9, 10, 10, 9]
[10, 9, 11]
[9, 8]
[5, 7, 8, 7]
[9, 7]
[5, 4, 6]
[12, 11, 12]
[9, 8, 10]
[8, 5]
[5, 3]
[3, 3, 4, 3]
[9, 8]
[3, 5, 4]
[7, 6]
[7, 7, 6, 6]
[9, 5]
[11, 10]
[10, 11, 8]
[9, 8, 6]
[7, 6, 4, 5, 5, 12]
[11, 8]
[9, 9, 8]
[10, 11, 12, 10]
[7, 10, 10, 9]
[4, 6, 5, 7]
[13, 11]
[5, 4]
[8, 7]
[10, 8, 7]
[12, 11, 10, 11, 10]
[10, 9, 10]
[5, 4]
[6, 5]
[10, 7, 8, 6, 8]
[10, 9]
[6, 3, 7, 5]
[4, 5, 11, 9, 10]
[8, 11, 8, 13]
[6, 7, 7, 6, 6]
[9, 8]
[8, 7]
[8, 7, 8, 7]
[9, 8]
[12, 11, 12, 13, 13]
[14, 13]
[6, 6, 4, 5]
[16, 15]
[9, 9, 11, 11, 10]
[11, 10, 8, 8, 7]
[10, 7, 9]
[8, 7]
[7, 9, 4, 4]
[7, 8, 7]
[10, 9]
[12, 10, 9, 9]
[9, 6]
[8, 6]
[11, 4]
[8, 7, 7]
[17, 16, 18, 17, 14, 13, 14, 4]
[7, 6]
[10, 4, 9, 10, 8, 12, 11]
[4, 3, 5]
[6, 7, 6]
[5, 4]
[9, 5, 6, 5, 6, 5]
[6, 7, 14, 15, 15, 9, 10, 9, 10]
[9, 8]
[14, 12]
[12, 13, 1

[11, 10]
[9, 10, 9]
[5, 4]
[6, 10, 4]
[9, 6]
[10, 9, 15]
[10, 8, 16]
[8, 7]
[8, 9, 10, 9, 6, 12, 13]
[11, 9]
[8, 7]
[5, 6, 4]
[6, 5]
[5, 6, 3]
[11, 8]
[7, 5]
[5, 5, 7, 7, 6]
[7, 9, 8, 8, 9]
[12, 11]
[6, 8, 7]
[10, 11, 11, 10]
[13, 12, 16]
[7, 6]
[8, 6]
[12, 11]
[7, 10, 7]
[6, 4, 5, 4, 15, 14, 13, 15]
[9, 9, 14, 11]
[14, 13]
[5, 4]
[7, 10, 8]
[10, 6]
[12, 14, 11]
[10, 12, 10]
[8, 8, 11, 8]
[4, 14, 12]
[9, 8, 4]
[7, 6]
[12, 11]
[4, 4, 5, 5, 4, 13]
[10, 8, 9, 11]
[6, 5]
[8, 7]
[7, 6]
[13, 12]
[12, 11, 10]
[5, 4]
[10, 9]
[11, 10, 9]
[6, 4]
[9, 8, 10, 9]
[7, 6]
[8, 6]
[11, 12, 3, 6, 6, 15, 16, 15, 16]
[9, 8, 8, 7]
[8, 8, 7]
[10, 9]
[6, 4]
[5, 4]
[9, 8]
[10, 8, 6]
[10, 9, 9, 9]
[9, 9, 8]
[10, 9, 9, 5]
[12, 15, 12]
[7, 6]
[9, 8]
[9, 8, 7, 6, 6]
[7, 8, 8, 7, 7, 8]
[8, 10, 8, 7, 9, 11, 5, 7, 9, 4, 6, 9]
[13, 8]
[9, 11, 10, 8]
[13, 12, 11, 10, 11, 11]
[11, 5]
[9, 9, 10, 8, 8, 9]
[8, 7, 8, 8]
[19, 14]
[7, 6]
[9, 10, 8]
[10, 9]
[4, 3]
[5, 4, 4]
[9, 5]
[3, 4, 8, 7]
[10, 8]
[8, 7]
[13, 10]
[8, 5]
[4

[8, 8, 10, 9, 7]
[8, 9, 9, 5]
[12, 3]
[11, 4, 7, 7, 5, 3, 11, 13, 15, 10, 12, 15]
[10, 9]
[9, 7, 8, 5, 6]
[10, 9]
[5, 4, 13, 14]
[8, 6]
[15, 16, 15, 16]
[13, 14, 3, 4]
[10, 10, 9]
[7, 3, 7]
[5, 4]
[11, 8, 9, 7, 9]
[10, 9]
[6, 5, 6]
[10, 5]
[12, 11]
[8, 5, 7]
[10, 12, 11]
[10, 10, 9]
[7, 6]
[11, 15, 13]
[7, 6]
[5, 4, 3, 4]
[4, 16, 15]
[5, 7, 5]
[13, 11]
[14, 13]
[11, 10, 12]
[5, 7, 6, 4]
[12, 3]
[7, 6, 6, 4, 5, 4, 3, 5, 3]
[8, 7, 7]
[11, 4, 11, 10]
[6, 5, 8, 8]
[8, 8, 8, 11, 12, 11, 11]
[7, 6, 3]
[8, 7]
[12, 11]
[11, 10]
[4, 3, 5, 10, 11, 10]
[9, 10, 10, 9]
[10, 10, 9]
[6, 6, 5]
[16, 12]
[10, 9]
[10, 9]
[11, 10]
[7, 5]
[7, 8, 8, 7]
[7, 9, 8, 7]
[6, 5]
[11, 10]
[12, 9, 10, 9, 8]
[7, 6]
[12, 11]
[5, 7, 6, 7, 5]
[8, 6]
[8, 10, 5, 4]
[13, 10, 11, 12]
[8, 7]
[11, 11, 8, 9, 8]
[15, 14, 14]
[12, 11]
[10, 10, 9, 8]
[11, 8]
[7, 6]
[7, 6]
[5, 4, 9]
[10, 8]
[5, 3]
[11, 9]
[13, 10, 11, 9, 6, 7]
[13, 13, 12, 10]
[6, 5]
[11, 10, 11, 12, 12]
[6, 5]
[10, 5]
[7, 6, 5, 7]
[7, 6]
[11, 10, 9, 10]
[11, 11, 

In [37]:
[len(d['paths']) for d in more_than_one_path_data]

[2,
 3,
 2,
 2,
 4,
 2,
 2,
 2,
 2,
 5,
 3,
 2,
 2,
 3,
 2,
 4,
 4,
 3,
 2,
 2,
 2,
 7,
 3,
 6,
 2,
 3,
 2,
 2,
 2,
 2,
 2,
 3,
 4,
 3,
 2,
 2,
 2,
 5,
 4,
 2,
 2,
 2,
 2,
 3,
 3,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 3,
 3,
 2,
 2,
 3,
 2,
 4,
 2,
 9,
 2,
 2,
 3,
 6,
 2,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 4,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 2,
 3,
 2,
 2,
 2,
 4,
 2,
 2,
 5,
 2,
 2,
 2,
 2,
 2,
 3,
 3,
 3,
 2,
 2,
 2,
 4,
 2,
 2,
 2,
 5,
 4,
 2,
 2,
 2,
 2,
 3,
 3,
 2,
 2,
 2,
 5,
 3,
 2,
 4,
 3,
 4,
 2,
 6,
 3,
 2,
 4,
 2,
 3,
 2,
 2,
 2,
 3,
 2,
 2,
 2,
 5,
 3,
 3,
 2,
 2,
 2,
 2,
 4,
 4,
 2,
 4,
 2,
 2,
 2,
 3,
 5,
 2,
 2,
 2,
 2,
 2,
 4,
 4,
 2,
 2,
 4,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 4,
 2,
 2,
 2,
 3,
 2,
 2,
 2,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 2,
 3,
 5,
 2,
 4,
 2,
 3,
 2,
 3,
 2,
 2,
 2,
 2,
 4,
 2,
 2,
 3,
 2,
 2,
 2,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 4,
 2,
 3,
 2,
 2,
 2,
 3,
 2,
 4,
 5,
 2,
 2,
 3,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 2,
 2,
 4,
 2,
 3,
 4,
 2,
 2,
 2,
 3,
 2,


In [38]:
[len(p) for p in more_than_one_path_data[4]['paths']]

[8, 8, 9, 9]

In [41]:
min_traj_len = lambda d: min([len(p) for p in d['paths']])
data_sorted = sorted(data, key=lambda x: min_traj_len(x))

In [43]:
[min_traj_len(d) for d in data]

[10,
 8,
 6,
 6,
 4,
 5,
 10,
 12,
 5,
 10,
 11,
 8,
 8,
 6,
 7,
 11,
 9,
 4,
 11,
 4,
 4,
 7,
 7,
 8,
 6,
 8,
 3,
 5,
 10,
 10,
 8,
 4,
 11,
 7,
 8,
 4,
 11,
 9,
 10,
 8,
 10,
 9,
 8,
 3,
 9,
 5,
 8,
 9,
 5,
 10,
 6,
 5,
 6,
 8,
 5,
 7,
 8,
 14,
 5,
 8,
 5,
 14,
 7,
 4,
 6,
 5,
 12,
 5,
 9,
 4,
 8,
 3,
 8,
 3,
 14,
 6,
 7,
 10,
 14,
 11,
 4,
 3,
 11,
 4,
 8,
 5,
 9,
 6,
 3,
 9,
 8,
 6,
 6,
 5,
 9,
 9,
 6,
 16,
 9,
 9,
 10,
 9,
 10,
 7,
 9,
 6,
 3,
 8,
 13,
 7,
 7,
 4,
 10,
 7,
 3,
 10,
 7,
 9,
 7,
 3,
 6,
 10,
 9,
 9,
 11,
 3,
 7,
 9,
 7,
 10,
 8,
 11,
 13,
 7,
 12,
 12,
 3,
 4,
 5,
 6,
 7,
 5,
 9,
 5,
 7,
 8,
 5,
 9,
 6,
 8,
 5,
 8,
 9,
 4,
 5,
 8,
 5,
 7,
 6,
 10,
 10,
 13,
 10,
 14,
 8,
 3,
 12,
 9,
 3,
 11,
 3,
 8,
 7,
 8,
 4,
 10,
 8,
 11,
 4,
 6,
 5,
 12,
 4,
 12,
 4,
 7,
 3,
 3,
 7,
 9,
 9,
 11,
 14,
 11,
 5,
 9,
 6,
 5,
 4,
 7,
 12,
 6,
 7,
 10,
 4,
 12,
 10,
 7,
 10,
 5,
 10,
 12,
 7,
 5,
 3,
 9,
 5,
 11,
 5,
 3,
 4,
 9,
 4,
 6,
 8,
 5,
 5,
 14,
 5,
 10,
 5,
 9,
 11,
 4,
 8,


In [44]:
[min_traj_len(d) for d in data_sorted]

[3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,


In [40]:
min([1,2,3])

1

In [33]:
import pickle
from collections import defaultdict

In [34]:
def nested_defaultdict():
    return defaultdict(list)

In [35]:
dum = defaultdict(nested_defaultdict)
dum[(1,2)] = {'b':[1,2,3]}
dum[(3,4)] = {'b':[1,2,3]}

In [36]:
dum[(5,6)] = {'d':[1,2,3]}

In [37]:
dum

defaultdict(<function __main__.nested_defaultdict()>,
            {(1, 2): {'b': [1, 2, 3]},
             (3, 4): {'b': [1, 2, 3]},
             (5, 6): {'d': [1, 2, 3]}})

In [40]:
with open("dummy.pickle", "wb") as handle:
    pickle.dump(dum, handle, protocol=pickle.HIGHEST_PROTOCOL)

PicklingError: Can't pickle <function nested_defaultdict at 0x7fe4783ac950>: it's not the same object as __main__.nested_defaultdict

In [22]:
dum[(2,3)] = {'c':[1,2,3]}

In [23]:
with open("dummy.pickle", "wb") as handle:
    pickle.dump(dum, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [39]:
def nested_defaultdict():
    return defaultdict(list)

_indexed_data = defaultdict(nested_defaultdict)
_indexed_data[(1,2)] = {'b':[1,2,3],'c':[1,2,3]}

In [25]:
with open("dummy.pickle", "wb") as handle:
    pickle.dump(_indexed_data, handle, protocol=pickle.HIGHEST_PROTOCOL)